# Install Packages

In [2]:
# !pip install -U scikit-image
# !pip install timm
# !pip install nibabel
# ! pip install python-gdcm
# ! pip install pylibjpeg pylibjpeg-libjpeg pydicom
# !pip install -U albumentations
# !pip install segmentation_models_pytorch
# !pip install -q git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached scikit_image-0.19.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (13.5 MB)
  Using cached tifffile-2021.11.2-py3-none-any.whl (178 kB)
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.16.2
    Uninstalling scikit-image-0.16.2:
      Successfully uninstalled scikit-image-0.16.2
Keyring is skipped due to an exception: 'keyring.backends'
  Using cached timm-0.6.11-py3-none-any.whl (548 kB)
  Using cached torchvision-0.14.0-cp37-cp37m-manylinux1_x86_64.whl (24.3 MB)
  Using cached torch-1.13.0-cp37-cp37m-manylinux1_x86_64.whl (890.2 MB)
  Using cached huggingface_hub-0.11.0-py3-none-any.whl (182 kB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using ca

# Stage 1

In [3]:
import os
import multiprocessing
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
datadir = '/root/input/rsna-2022-cervical-spine-fracture-detection'
libdir = '/root/workspace/RSNA2022RAWE'
outputdir = '/root/workspace/RSNA2022RAWE'
otherdir = '/root/workspace/RSNA2022RAWE'

train_bs_ = 16 # train_batch_size
valid_bs_ = 128 # valid_batch_size
num_workers_ = multiprocessing.cpu_count()

In [4]:
class CFG:
    seed=42
    device='GPU' # ['TPU', 'GPU']
    nprocs=1 # [1, 8]
    num_workers=num_workers_
    train_bs=train_bs_
    valid_bs=valid_bs_
    fold_num=5 
    
    target_cols=["C1", "C2", "C3", "C4", "C5", "C6", "C7", "OT"]
    num_classes=8 
    
    normalize_mean=[0.4824, 0.4824, 0.4824] 
    normalize_std=[0.22, 0.22, 0.22] 
    
    fold_list=[0]

    model_arch="efficientnet-b0" 
    img_size=320 
    croped_img_size = 320 # 裁剪后的图片尺寸
    weight_path = f"{outputdir}/efficientnet-b0_109_fold0_epoch9.pth" 

In [5]:
import sys; 
    
import ast
from glob import glob
import cv2
from skimage import io
import os
from datetime import datetime
import time
import random
from tqdm import tqdm
from contextlib import contextmanager
import math

import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
import torch
import torchvision
from torchvision import transforms
from torch import nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch.optim import Adam, SGD, AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from warmup_scheduler import GradualWarmupScheduler
import timm
import warnings
import joblib
from scipy.ndimage.interpolation import zoom
import nibabel as nib
import pydicom as dicom
import gc 


if CFG.device == 'TPU':
    !pip install -q pytorch-ignite
    import ignite.distributed as idist
elif CFG.device == 'GPU':
    from torch.cuda.amp import autocast, GradScaler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
print(torch.version.cuda)

11.7


In [7]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True 

seed_everything(CFG.seed)


def get_score(y_true, y_pred):
    scores = []
    for i in range(y_true.shape[1]):
        score = roc_auc_score(y_true[:,i], y_pred[:,i])
        scores.append(score)
    avg_score = np.mean(scores)
    return avg_score, scores



def init_logger(log_file=outputdir+'/train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger




def get_timediff(time1,time2):
    minute_,second_ = divmod(time2-time1,60)
    return f"{int(minute_):02d}:{int(second_):02d}"  


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb


def load_dicom(path):
    """
    This supports loading both regular and compressed JPEG images. 
    See the first sell with `pip install` commands for the necessary dependencies
    """
    img = dicom.dcmread(path)
    img.PhotometricInterpretation = 'YBR_FULL'
    data = img.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    # data = (data * 255).astype(np.uint8)
    return data

# load the data

In [8]:
ss = 'input/rsna-2022-cervical-spine-fracture-detection/sample_submission.csv'
# Fix mismatch with test_images folder
test_df = pd.DataFrame(columns = ['row_id','StudyInstanceUID','prediction_type'])
for i in ['1.2.826.0.1.3680043.22327','1.2.826.0.1.3680043.25399','1.2.826.0.1.3680043.5876']:
    for j in ['C1','C2','C3','C4','C5','C6','C7','patient_overall']:
        test_df = test_df.append({'row_id':i+'_'+j,'StudyInstanceUID':i,'prediction_type':j},ignore_index=True)

# Sample submission
ss = pd.DataFrame(test_df['row_id'])
ss['fractured'] = 0.5

display(test_df.head(3))

,row_id,StudyInstanceUID,prediction_type
0,1.2.826.0.1.3680043.22327_C1,1.2.826.0.1.3680043.22327,C1
1,1.2.826.0.1.3680043.22327_C2,1.2.826.0.1.3680043.22327,C2
2,1.2.826.0.1.3680043.22327_C3,1.2.826.0.1.3680043.22327,C3


In [9]:
test_slice_list = []
for file_name in tqdm(test_df["StudyInstanceUID"].values):
    test_image_path = glob(f"{datadir}/test_images/{file_name}/*")
    test_image_path = sorted(test_image_path, key=lambda x:int(x.split("/")[-1].replace(".dcm","")))
    for path_idx in range(len(test_image_path)):
        path1 = "nofile" if path_idx-1 < 0 else test_image_path[path_idx-1].replace(f"{datadir}/", "")
        path2 = test_image_path[path_idx].replace(f"{datadir}/", "")
        path3 = "nofile" if path_idx+1 >= len(test_image_path) else test_image_path[path_idx+1].replace(f"{datadir}/", "")
        slice_num = int(path2.split("/")[-1].replace(".dcm",""))
        test_slice_list.append([f"{file_name}_{slice_num}", file_name, slice_num, path1, path2, path3])

100%|██████████| 24/24 [00:00<00:00, 72.95it/s]


In [10]:
test_df = pd.DataFrame(test_slice_list, columns=["id", "StudyInstanceUID", "slice_num", "path1", "path2", "path3"])
test_df = test_df.sort_values(['StudyInstanceUID', 'slice_num'], ascending = [True, True]).reset_index(drop=True)
test_df.to_csv(f'{outputdir}/test_slice_list.csv', index=False)
test_df

,id,StudyInstanceUID,slice_num,path1,path2,path3
0,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
1,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
2,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
3,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
4,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
...,...,...,...,...,...,...
10539,1.2.826.0.1.3680043.5876_458,1.2.826.0.1.3680043.5876,458,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,nofile
10540,1.2.826.0.1.3680043.5876_458,1.2.826.0.1.3680043.5876,458,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,nofile
10541,1.2.826.0.1.3680043.5876_458,1.2.826.0.1.3680043.5876,458,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,nofile
10542,1.2.826.0.1.3680043.5876_458,1.2.826.0.1.3680043.5876,458,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,nofile


In [11]:
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        im2 = load_dicom(f"{datadir}/{row['path2']}")   # 512*512  
        im2h = im2.shape[0]
        im2w = im2.shape[1]

        im1 = load_dicom(f"{datadir}/{row['path1']}") if row['path1'] != "nofile" else np.zeros((im2h, im2w))  # 512*512                                                       
        im3 = load_dicom(f"{datadir}/{row['path3']}") if row['path3'] != "nofile" else np.zeros((im2h, im2w))  # 512*512  

        if im1.shape !=  (im2h, im2w):
            im1 = cv2.resize(im1, (im2w, im2h))
        if im3.shape !=  (im2h, im2w):
            im3 = cv2.resize(im3, (im2w, im2h)) 
        image_list = [im1, im2, im3]
        image = np.stack(image_list, axis=2) # 512*512*3; 0-1

        # transform
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        # image = image/255.0
        image = np.transpose(image, (2, 0, 1)) # 3*img_size*img_size; 0-1
        return torch.from_numpy(image), row['StudyInstanceUID'], row['slice_num'] 

In [12]:
# augmentation
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, 
    CenterCrop, Resize, RandomCrop, GaussianBlur, JpegCompression, Downscale, ElasticTransform
)
import albumentations

from albumentations.pytorch import ToTensorV2

def get_transforms(data):
    if data == 'valid':
        return Compose([
            Resize(CFG.img_size, CFG.img_size, interpolation=cv2.INTER_NEAREST),
        ])

In [13]:
import segmentation_models_pytorch as smp

def build_model():
    model = smp.Unet(
        encoder_name=CFG.model_arch,    # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=CFG.num_classes,        # model output channels (number of classes in your dataset)
        activation=None,
    )
    model.to(device)
    return model

def load_model(path):
    model = build_model()
    model.load_state_dict(torch.load(path)["model"])
    model.eval()
    return model

In [14]:
os.makedirs(f"{outputdir}/test_voxel_mask", exist_ok=True)

In [15]:
vertebra_class = []
voxel_crop_list = []
def crop_voxel(voxels, voxel_mask, last_f_name, croped_img_size):
    area_thr = 10
    # x
    x_list = []
    length = voxel_mask.shape[0]
    for i in range(length):
        if voxel_mask[i].sum().item() >= area_thr:
            x_list.append(i)
            break
    else:
        x_list.append(0)

    for i in range(length-1, -1, -1):
        if voxel_mask[i].sum().item() >= area_thr:
            x_list.append(i)
            break
    else:
        x_list.append(length-1)

    # y
    y_list = []
    length = voxel_mask.shape[1]
    for i in range(length):
        if voxel_mask[:, i].sum().item() >= area_thr:
            y_list.append(i)
            break
    else:
        y_list.append(0)

    for i in range(length-1, -1, -1):
        if voxel_mask[:, i].sum().item() >= area_thr:
            y_list.append(i)
            break
    else:
        y_list.append(length-1)

    # z
    z_list = []
    length = voxel_mask.shape[2]
    for i in range(length):
        if voxel_mask[:, :, i].sum().item() >= area_thr:
            z_list.append(i)
            break
    else:
        z_list.append(0)

    for i in range(length-1, -1, -1):
        if voxel_mask[:, :, i].sum().item() >= area_thr:
            z_list.append(i)
            break
    else:
        z_list.append(length-1)
        
    try:
        croped_voxel_mask = voxel_mask[x_list[0]:x_list[1]+1, y_list[0]:y_list[1]+1, z_list[0]:z_list[1]+1]
        row = [last_f_name, voxel_mask.shape[1], x_list[0], x_list[1]+1, y_list[0], y_list[1]+1, z_list[0], z_list[1]+1]
    except:
        print(f"last_f_name:{last_f_name}, voxel_mask.shape:{voxel_mask.shape}, x_list:{x_list}, y_list:{y_list}, z_list:{z_list}")
        croped_voxel_mask = voxel_mask
        row = [last_f_name, voxel_mask.shape[1], 0, voxel_mask.shape[0], 0, voxel_mask.shape[1], 0, voxel_mask.shape[2]]
    voxel_crop_list.append(row)

    croped_voxel_mask = croped_voxel_mask.to('cpu').numpy().astype(np.uint8) # bs*img_size*img_size; 0-8 classes


    for x_idx in range(croped_voxel_mask.shape[0]):
        slice_mask = croped_voxel_mask[x_idx]

        unique, counts = np.unique(slice_mask, return_counts=True)
        if len(unique) == 1 and unique[0] == 0:
            vertebra_class.append([last_f_name, x_idx, 0])
        elif unique[0] == 0:
            unique = unique[1:]
            counts = counts[1:]
            vertebra_class.append([last_f_name, x_idx, unique[counts.argmax()]])
        else:
            vertebra_class.append([last_f_name, x_idx, unique[counts.argmax()]])
        
    return None, croped_voxel_mask

In [16]:

test_dataset = TestDataset(test_df, transform=get_transforms("valid")) # get_transforms("valid")
test_loader = DataLoader(test_dataset, batch_size=CFG.valid_bs, num_workers=CFG.num_workers, shuffle=False, pin_memory=True, drop_last=False)

model = load_model(CFG.weight_path)
model.eval()
last_f_name = ""
voxel_mask = []
voxels = []
for step, (images, file_names, n_slice) in tqdm(enumerate(test_loader),total=len(test_loader)):
    images = images.to(device, dtype=torch.float) # bs*3*image_size*image_size
    batch_size = images.size(0)
    with torch.no_grad():
        y_pred = model(images) # [B, 8, H, W]
    y_pred = y_pred.sigmoid() ####
    slice_mask_max = torch.max(y_pred, 1) # bs*img_size*img_size
    slice_mask = torch.where((slice_mask_max.values)>0.5, slice_mask_max.indices+1, 0) # bs*img_size*img_size; 0-8 classes

    start_idx = 0
    for bs_idx in range(batch_size):
        f_name = file_names[bs_idx]
        if f_name != last_f_name:
            voxel_mask.append(slice_mask[start_idx:bs_idx])
            voxel_mask = torch.cat(voxel_mask, dim=0) # n_slice*img_size*img_size; 0-8 classes
            
            if len(voxel_mask) > 0:
                croped_voxel, croped_voxel_mask = crop_voxel(None, voxel_mask, last_f_name, CFG.croped_img_size)
            last_f_name = f_name
            start_idx = bs_idx
            voxel_mask = []
            # voxels = []
        elif bs_idx == batch_size-1:
            voxel_mask.append(slice_mask[start_idx:batch_size])

voxel_mask = torch.cat(voxel_mask, dim=0)
if len(voxel_mask) > 0:
    croped_voxel, croped_voxel_mask = crop_voxel(None, voxel_mask, last_f_name, CFG.croped_img_size)
torch.cuda.empty_cache()
gc.collect()

100%|██████████| 83/83 [01:43<00:00,  1.24s/it]


8

In [17]:
len(voxel_crop_list) # 2019

3

In [18]:
study_crop_df = pd.DataFrame(voxel_crop_list, columns=["StudyInstanceUID", "before_image_size", "x0", "x1", "y0", "y1", "z0", "z1"])
study_crop_df.to_csv(f"{datadir}/test_study_crop_info.csv", index=False)
study_crop_df

,StudyInstanceUID,before_image_size,x0,x1,y0,y1,z0,z1
0,1.2.826.0.1.3680043.22327,320,0,2496,6,265,86,238
1,1.2.826.0.1.3680043.25399,320,784,4336,109,274,65,204
2,1.2.826.0.1.3680043.5876,320,0,3656,3,249,71,225


In [19]:
test_slice_list = pd.read_csv(f"{outputdir}/test_slice_list.csv")
test_slice_list

,id,StudyInstanceUID,slice_num,path1,path2,path3
0,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
1,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
2,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
3,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
4,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
...,...,...,...,...,...,...
10539,1.2.826.0.1.3680043.5876_458,1.2.826.0.1.3680043.5876,458,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,nofile
10540,1.2.826.0.1.3680043.5876_458,1.2.826.0.1.3680043.5876,458,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,nofile
10541,1.2.826.0.1.3680043.5876_458,1.2.826.0.1.3680043.5876,458,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,nofile
10542,1.2.826.0.1.3680043.5876_458,1.2.826.0.1.3680043.5876,458,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,nofile


In [20]:
new_df = []
for idx, study_id, _, x0, x1, _, _, _, _, in study_crop_df.itertuples():
    one_study = test_slice_list[test_slice_list["StudyInstanceUID"] == study_id].reset_index(drop=True)
    new_df.append(one_study[x0:x1])
new_df = pd.concat(new_df, axis=0).reset_index(drop=True)
new_df

,id,StudyInstanceUID,slice_num,path1,path2,path3
0,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
1,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
2,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
3,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
4,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm
...,...,...,...,...,...,...
9699,1.2.826.0.1.3680043.5876_457,1.2.826.0.1.3680043.5876,457,test_images/1.2.826.0.1.3680043.5876/456.dcm,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm
9700,1.2.826.0.1.3680043.5876_457,1.2.826.0.1.3680043.5876,457,test_images/1.2.826.0.1.3680043.5876/456.dcm,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm
9701,1.2.826.0.1.3680043.5876_457,1.2.826.0.1.3680043.5876,457,test_images/1.2.826.0.1.3680043.5876/456.dcm,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm
9702,1.2.826.0.1.3680043.5876_457,1.2.826.0.1.3680043.5876,457,test_images/1.2.826.0.1.3680043.5876/456.dcm,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm


In [21]:
slice_class_df = pd.DataFrame(vertebra_class, columns=["StudyInstanceUID", "new_slice_num", "vertebra_class"])
slice_class_df.sort_values(by=["StudyInstanceUID", "new_slice_num"], inplace=True)
slice_class_df

,StudyInstanceUID,new_slice_num,vertebra_class
0,1.2.826.0.1.3680043.22327,0,2
1,1.2.826.0.1.3680043.22327,1,2
2,1.2.826.0.1.3680043.22327,2,2
3,1.2.826.0.1.3680043.22327,3,2
4,1.2.826.0.1.3680043.22327,4,2
...,...,...,...
9699,1.2.826.0.1.3680043.5876,3651,8
9700,1.2.826.0.1.3680043.5876,3652,8
9701,1.2.826.0.1.3680043.5876,3653,8
9702,1.2.826.0.1.3680043.5876,3654,8


In [22]:
new_df = pd.concat([new_df, slice_class_df[["new_slice_num", "vertebra_class"]]], axis=1)
new_df

,id,StudyInstanceUID,slice_num,path1,path2,path3,new_slice_num,vertebra_class
0,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm,0,2
1,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm,1,2
2,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm,2,2
3,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm,3,2
4,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm,4,2
...,...,...,...,...,...,...,...,...
9699,1.2.826.0.1.3680043.5876_457,1.2.826.0.1.3680043.5876,457,test_images/1.2.826.0.1.3680043.5876/456.dcm,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,3651,8
9700,1.2.826.0.1.3680043.5876_457,1.2.826.0.1.3680043.5876,457,test_images/1.2.826.0.1.3680043.5876/456.dcm,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,3652,8
9701,1.2.826.0.1.3680043.5876_457,1.2.826.0.1.3680043.5876,457,test_images/1.2.826.0.1.3680043.5876/456.dcm,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,3653,8
9702,1.2.826.0.1.3680043.5876_457,1.2.826.0.1.3680043.5876,457,test_images/1.2.826.0.1.3680043.5876/456.dcm,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,3654,8


In [23]:
new_slice_df = new_df.merge(study_crop_df, on="StudyInstanceUID", how="left")
new_slice_df

,id,StudyInstanceUID,slice_num,path1,path2,path3,new_slice_num,vertebra_class,before_image_size,x0,x1,y0,y1,z0,z1
0,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm,0,2,320,0,2496,6,265,86,238
1,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm,1,2,320,0,2496,6,265,86,238
2,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm,2,2,320,0,2496,6,265,86,238
3,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm,3,2,320,0,2496,6,265,86,238
4,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,nofile,test_images/1.2.826.0.1.3680043.22327/1.dcm,test_images/1.2.826.0.1.3680043.22327/2.dcm,4,2,320,0,2496,6,265,86,238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9699,1.2.826.0.1.3680043.5876_457,1.2.826.0.1.3680043.5876,457,test_images/1.2.826.0.1.3680043.5876/456.dcm,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,3651,8,320,0,3656,3,249,71,225
9700,1.2.826.0.1.3680043.5876_457,1.2.826.0.1.3680043.5876,457,test_images/1.2.826.0.1.3680043.5876/456.dcm,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,3652,8,320,0,3656,3,249,71,225
9701,1.2.826.0.1.3680043.5876_457,1.2.826.0.1.3680043.5876,457,test_images/1.2.826.0.1.3680043.5876/456.dcm,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,3653,8,320,0,3656,3,249,71,225
9702,1.2.826.0.1.3680043.5876_457,1.2.826.0.1.3680043.5876,457,test_images/1.2.826.0.1.3680043.5876/456.dcm,test_images/1.2.826.0.1.3680043.5876/457.dcm,test_images/1.2.826.0.1.3680043.5876/458.dcm,3654,8,320,0,3656,3,249,71,225


In [24]:
new_slice_df.to_csv(f"{datadir}/test_slice.csv", index=False)

In [25]:
sample_num = 24
vertebrae_df_list = []
for study_id in tqdm(np.unique(new_slice_df["StudyInstanceUID"])):
    one_study = new_slice_df[new_slice_df["StudyInstanceUID"] == study_id].reset_index(drop=True)
    for cid in range(1, 8):
        one_study_cid = one_study[one_study["vertebra_class"] == cid].reset_index(drop=True)
        if len(one_study_cid) >= sample_num:
            sample_index = np.linspace(0, len(one_study_cid)-1, sample_num, dtype=int)
            one_study_cid = one_study_cid.iloc[sample_index].reset_index(drop=True)
        if len(one_study_cid) < 5:
            continue
        slice_num_list = one_study_cid["slice_num"].values.tolist()
        arow = one_study_cid.iloc[0]
        vertebrae_df_list.append([f"{study_id}_{cid}", study_id, cid, slice_num_list, arow["before_image_size"], \
            arow["x0"], arow["x1"], arow["y0"], arow["y1"], arow["z0"], arow["z1"]])

100%|██████████| 3/3 [00:00<00:00, 113.71it/s]


In [26]:
vertebrae_df = pd.DataFrame(vertebrae_df_list, columns=["study_cid", "StudyInstanceUID", "cid", "slice_num_list", \
    "before_image_size", "x0", "x1", "y0", "y1", "z0", "z1"])
vertebrae_df.to_pickle(f"{datadir}/vertebrae_df_test.pkl")    
vertebrae_df

,study_cid,StudyInstanceUID,cid,slice_num_list,before_image_size,x0,x1,y0,y1,z0,z1
0,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,"[46, 47, 49, 50, 52, 54, 55, 57, 58, 60, 62, 6...",320,0,2496,6,265,86,238
1,1.2.826.0.1.3680043.22327_2,1.2.826.0.1.3680043.22327,2,"[1, 83, 84, 86, 87, 89, 90, 92, 93, 95, 96, 98...",320,0,2496,6,265,86,238
2,1.2.826.0.1.3680043.22327_3,1.2.826.0.1.3680043.22327,3,"[116, 117, 118, 119, 120, 121, 122, 123, 125, ...",320,0,2496,6,265,86,238
3,1.2.826.0.1.3680043.22327_4,1.2.826.0.1.3680043.22327,4,"[142, 143, 144, 145, 146, 147, 148, 149, 150, ...",320,0,2496,6,265,86,238
4,1.2.826.0.1.3680043.22327_5,1.2.826.0.1.3680043.22327,5,"[167, 167, 168, 169, 170, 171, 172, 173, 174, ...",320,0,2496,6,265,86,238
5,1.2.826.0.1.3680043.22327_6,1.2.826.0.1.3680043.22327,6,"[189, 190, 192, 193, 194, 195, 196, 197, 198, ...",320,0,2496,6,265,86,238
6,1.2.826.0.1.3680043.22327_7,1.2.826.0.1.3680043.22327,7,"[215, 216, 217, 218, 219, 220, 221, 222, 224, ...",320,0,2496,6,265,86,238
7,1.2.826.0.1.3680043.25399_1,1.2.826.0.1.3680043.25399,1,"[99, 101, 103, 105, 107, 109, 111, 113, 116, 1...",320,784,4336,109,274,65,204
8,1.2.826.0.1.3680043.25399_2,1.2.826.0.1.3680043.25399,2,"[148, 150, 152, 154, 156, 158, 160, 162, 165, ...",320,784,4336,109,274,65,204
9,1.2.826.0.1.3680043.25399_3,1.2.826.0.1.3680043.25399,3,"[197, 198, 200, 202, 203, 205, 207, 209, 210, ...",320,784,4336,109,274,65,204


# Stage 2

In [27]:
import os
import multiprocessing
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

datadir = '/root/input/rsna-2022-cervical-spine-fracture-detection'
libdir = '/root/workspace/RSNA2022RAWE'
outputdir = '/root/workspace/RSNA2022RAWE'
otherdir = '/root/workspace/RSNA2022RAWE'
train_bs_ = 4
valid_bs_ = 8
num_workers_ = multiprocessing.cpu_count()

In [28]:
class CFG:
    seed=42
    device='GPU'
    nprocs=1 # [1, 8]
    num_workers=num_workers_
    train_bs=train_bs_
    valid_bs=valid_bs_
    fold_num=5

    target_cols=["C1", "C2", "C3", "C4", "C5", "C6", "C7"]
    num_classes=7

    accum_iter=1
    max_grad_norm=1000
    print_freq=100
    normalize_mean=[0.4824, 0.4824, 0.4824] # [0.485, 0.456, 0.406] [0.4824, 0.4824, 0.4824]
    normalize_std=[0.22, 0.22, 0.22] # [0.229, 0.224, 0.225] [0.22, 0.22, 0.22]
    
    suffix="401" 
    fold_list=[0] 
    epochs=25
    model_arch="resnet50d" # tf_efficientnetv2_s, resnest50d
    img_size=320
    optimizer="AdamW"
    scheduler="CosineAnnealingLR"
    loss_fn="BCEWithLogitsLoss"
    scheduler_warmup="GradualWarmupSchedulerV3" 

    warmup_epo=1
    warmup_factor = 10
    T_max= epochs-warmup_epo-2 if scheduler_warmup=="GradualWarmupSchedulerV2" else \
           epochs-warmup_epo-1 if scheduler_warmup=="GradualWarmupSchedulerV3" else epochs-1 # CosineAnnealingLR
    
    seq_len = 24
    lr=5e-4
    min_lr=1e-6 
    weight_decay=0
    dropout=0.1

    gpu_parallel=False
    n_early_stopping=4
    debug=False
    multihead=False

# Import

In [29]:
import sys; 
    
import ast
from glob import glob
import cv2
from skimage import io
import os
from datetime import datetime
import time
import random
from tqdm import tqdm
from contextlib import contextmanager
import math

import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
from sklearn.model_selection import GroupKFold, StratifiedKFold
import torch
import torchvision
from torchvision import transforms
from torch import nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch.optim import Adam, SGD, AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from warmup_scheduler import GradualWarmupScheduler
import timm
import warnings
import joblib
from scipy.ndimage.interpolation import zoom
import nibabel as nib
import pydicom as dicom
import gc 
from torch.nn import DataParallel



if CFG.device == 'TPU':
    !pip install -q pytorch-ignite
    import ignite.distributed as idist
elif CFG.device == 'GPU':
    from torch.cuda.amp import autocast, GradScaler

# helper

In [30]:
test_df = pd.read_pickle(f'{datadir}/vertebrae_df_test.pkl')
submission_df = pd.read_csv(f'{datadir}/sample_submission.csv')

In [31]:
test_df.head()

,study_cid,StudyInstanceUID,cid,slice_num_list,before_image_size,x0,x1,y0,y1,z0,z1
0,1.2.826.0.1.3680043.22327_1,1.2.826.0.1.3680043.22327,1,"[46, 47, 49, 50, 52, 54, 55, 57, 58, 60, 62, 6...",320,0,2496,6,265,86,238
1,1.2.826.0.1.3680043.22327_2,1.2.826.0.1.3680043.22327,2,"[1, 83, 84, 86, 87, 89, 90, 92, 93, 95, 96, 98...",320,0,2496,6,265,86,238
2,1.2.826.0.1.3680043.22327_3,1.2.826.0.1.3680043.22327,3,"[116, 117, 118, 119, 120, 121, 122, 123, 125, ...",320,0,2496,6,265,86,238
3,1.2.826.0.1.3680043.22327_4,1.2.826.0.1.3680043.22327,4,"[142, 143, 144, 145, 146, 147, 148, 149, 150, ...",320,0,2496,6,265,86,238
4,1.2.826.0.1.3680043.22327_5,1.2.826.0.1.3680043.22327,5,"[167, 167, 168, 169, 170, 171, 172, 173, 174, ...",320,0,2496,6,265,86,238


In [32]:
if CFG.device == 'TPU':
    import os
    VERSION = "1.7"
    CP_V = "36" if ENV == "colab" else "37"
    wheel = f"torch_xla-{VERSION}-cp{CP_V}-cp{CP_V}m-linux_x86_64.whl"
    url = f"https://storage.googleapis.com/tpu-pytorch/wheels/{wheel}"
    !pip3 -q install cloud-tpu-client==0.10 $url
    os.system('export XLA_USE_BF16=1')
    import torch_xla.core.xla_model as xm
    import torch_xla.distributed.parallel_loader as pl
    import torch_xla.distributed.xla_multiprocessing as xmp
    CFG.lr = CFG.lr * CFG.nprocs
    CFG.train_bs = CFG.train_bs // CFG.nprocs
    device = xm.xla_device()
    
elif CFG.device == "GPU":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [33]:
print(device)

cuda


In [34]:
print(torch.version.cuda)

11.7


# DataSet

In [35]:
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        study_id = row["StudyInstanceUID"]
        slice_num_list = row['slice_num_list']
        slice_list = []
        for s_num in slice_num_list:
            path = f"{datadir}/test_images/{study_id}/{s_num}.dcm"
            img = load_dicom(path)
            if len(slice_list) == 0:
                imgh = img.shape[0]
                imgw = img.shape[1]
            elif img.shape != (imgh, imgw):
                img = cv2.resize(img,(imgh,imgw))

            slice_list.append(img)
        for _ in range(CFG.seq_len - len(slice_list)):
            slice_list.append(np.zeros((imgh,imgw)))

        image = np.stack(slice_list, axis=2) # 512*512*seq_len; 0-1

        assert image.shape == (imgh, imgw, CFG.seq_len)

        # transform
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        image = np.transpose(image, (2, 0, 1)) # seq_len*img_size*img_size; 0-1
        return torch.from_numpy(image)

In [36]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, 
    CenterCrop, Resize, RandomCrop, GaussianBlur, JpegCompression, Downscale, ElasticTransform
)
import albumentations
from albumentations.pytorch import ToTensorV2

def get_transforms(*, data):
    if data == 'valid':
        return Compose([
            Resize(CFG.img_size, CFG.img_size),
        ])

In [37]:
from pylab import rcParams
dataset_show = TestDataset(
    test_df,
    transform=get_transforms(data='light_train') # None, get_transforms(data='check')
    )
rcParams['figure.figsize'] = 30,20

for i in range(4):
    f, axarr = plt.subplots(1,5)
    idx = np.random.randint(0, len(dataset_show))
    img= dataset_show[idx]
    # axarr[p].imshow(img) # transform=None
    axarr[0].imshow(img[0]); plt.axis('OFF');
    axarr[1].imshow(img[1]); plt.axis('OFF');
    axarr[2].imshow(img[2]); plt.axis('OFF');
    axarr[3].imshow(img[3]); plt.axis('OFF');
    axarr[4].imshow(img[4]); plt.axis('OFF');

# Model

In [38]:
import torch.nn as nn
from itertools import repeat

class SpatialDropout(nn.Module):
    def __init__(self, drop=0.5):
        super(SpatialDropout, self).__init__()
        self.drop = drop
        
    def forward(self, inputs, noise_shape=None):
        """
        @param: inputs, tensor
        @param: noise_shape, tuple
        """
        outputs = inputs.clone()
        if noise_shape is None:
            noise_shape = (inputs.shape[0], *repeat(1, inputs.dim()-2), inputs.shape[-1]) 
        
        self.noise_shape = noise_shape
        if not self.training or self.drop == 0:
            return inputs
        else:
            noises = self._make_noises(inputs)
            if self.drop == 1:
                noises.fill_(0.0)
            else:
                noises.bernoulli_(1 - self.drop).div_(1 - self.drop)
            noises = noises.expand_as(inputs)    
            outputs.mul_(noises)
            return outputs
            
    def _make_noises(self, inputs):
        return inputs.new().resize_(self.noise_shape)


import torch
from torch import nn
import torch.nn.functional as F

from typing import Dict, Optional
 
import numpy as np
import torch
import torch.nn.functional as F
from torch import Tensor


    
class MLPAttentionNetwork(nn.Module):
 
    def __init__(self, hidden_dim, attention_dim=None):
        super(MLPAttentionNetwork, self).__init__()
 
        self.hidden_dim = hidden_dim
        self.attention_dim = attention_dim
        if self.attention_dim is None:
            self.attention_dim = self.hidden_dim
        # W * x + b
        self.proj_w = nn.Linear(self.hidden_dim, self.attention_dim, bias=True)
        # v.T
        self.proj_v = nn.Linear(self.attention_dim, 1, bias=False)
 
    def forward(self, x):
        """
        :param x: seq_len, batch_size, hidden_dim
        :return: batch_size * seq_len, batch_size * hidden_dim
        """
        batch_size, seq_len, _ = x.size()
        
        H = torch.tanh(self.proj_w(x)) # (batch_size, seq_len, hidden_dim)
        
        att_scores = torch.softmax(self.proj_v(H),axis=1) # (batch_size, seq_len)
        
        attn_x = (x * att_scores).sum(1) # (batch_size, hidden_dim)
        return attn_x

In [39]:
class RSNAClassifier(nn.Module):
    def __init__(self, model_arch, hidden_dim=256, seq_len=24, pretrained=False):
        super().__init__()
        self.seq_len = seq_len
        self.model = timm.create_model(model_arch, in_chans=1, pretrained=pretrained)

        if 'efficientnet' in CFG.model_arch:
            cnn_feature = self.model.classifier.in_features
            self.model.classifier = nn.Identity()
        elif "res" in CFG.model_arch:
            cnn_feature = self.model.fc.in_features
            self.model.global_pool = nn.Identity()
            self.model.fc = nn.Identity()
            self.pooling = nn.AdaptiveAvgPool2d(1)
        
        self.spatialdropout = SpatialDropout(CFG.dropout)
        self.gru = nn.GRU(cnn_feature, hidden_dim, 2, batch_first=True, bidirectional=True)
        self.mlp_attention_layer = MLPAttentionNetwork(2 * hidden_dim)
        self.logits = nn.Sequential(
            nn.Linear(hidden_dim*2, 128),
            nn.ReLU(),
            nn.Dropout(CFG.dropout),
            nn.Linear(128, 1)
        )

        for n, m in self.named_modules():
            if isinstance(m, nn.GRU):
                print(f"init {m}")
                for param in m.parameters():
                    if len(param.shape) >= 2:
                        nn.init.orthogonal_(param.data)
                    else:
                        nn.init.normal_(param.data)

    def forward(self, x): # (B, seq_len, H, W)
        bs = x.size(0) 
        x = x.reshape(bs*self.seq_len, 1, x.size(2), x.size(3)) # (B*seq_len, 1, H, W)
        features = self.model(x)   
        if "res" in CFG.model_arch:                             
            features = self.pooling(features).view(bs*self.seq_len, -1) # (B*seq_len, cnn_feature)
        features = self.spatialdropout(features)                # (B*seq_len, cnn_feature)
        # print(features.shape)
        features = features.reshape(bs, self.seq_len, -1)       # (B, seq_len, cnn_feature)
        features, _ = self.gru(features)                        # (B, seq_len, hidden_dim*2)
        atten_out = self.mlp_attention_layer(features)          # (B, hidden_dim*2)
        pred = self.logits(atten_out)                           # (B, 1)
        pred = pred.view(bs, -1)                                # (B, 1)
        return pred

In [40]:
model = RSNAClassifier(CFG.model_arch, hidden_dim=256, seq_len=24, pretrained=True)

init GRU(2048, 256, num_layers=2, batch_first=True, bidirectional=True)


In [41]:
def get_activation(activ_name: str="relu"):
    """"""
    act_dict = {
        "relu": nn.ReLU(inplace=True),
        "tanh": nn.Tanh(),
        "sigmoid": nn.Sigmoid(),
        "identity": nn.Identity()}
    if activ_name in act_dict:
        return act_dict[activ_name]
    else:
        raise NotImplementedError
        

class Conv2dBNActiv(nn.Module):
    """Conv2d -> (BN ->) -> Activation"""
    
    def __init__(
        self, in_channels, out_channels,
        kernel_size, stride, padding,
        bias=False, use_bn=True, activ="relu"
    ):
        """"""
        super(Conv2dBNActiv, self).__init__()
        layers = []
        layers.append(nn.Conv2d(
            in_channels, out_channels,
            kernel_size, stride, padding, bias=bias))
        if use_bn:
            layers.append(nn.BatchNorm2d(out_channels))
            
        layers.append(get_activation(activ))
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        """Forward"""
        return self.layers(x)
        
    
class SpatialAttentionBlock(nn.Module):
    """Spatial Attention for (C, H, W) feature maps"""
    
    def __init__(
        self, in_channels,
        out_channels_list,
    ):
        """Initialize"""
        super(SpatialAttentionBlock, self).__init__()
        self.n_layers = len(out_channels_list)
        channels_list = [in_channels] + out_channels_list
        assert self.n_layers > 0
        assert channels_list[-1] == 1
        
        for i in range(self.n_layers - 1):
            in_chs, out_chs = channels_list[i: i + 2]
            layer = Conv2dBNActiv(in_chs, out_chs, 3, 1, 1, activ="relu")
            setattr(self, f"conv{i + 1}", layer)
            
        in_chs, out_chs = channels_list[-2:]
        layer = Conv2dBNActiv(in_chs, out_chs, 3, 1, 1, activ="sigmoid")
        setattr(self, f"conv{self.n_layers}", layer)
    
    def forward(self, x):
        """Forward"""
        h = x
        for i in range(self.n_layers):
            h = getattr(self, f"conv{i + 1}")(h)
            
        h = h * x
        return h



class MultiHeadResNet200D(nn.Module):
    def __init__(self, out_dims_head=[3, 4, 3, 1],  pretrained=False):
        self.base_name = "resnet200d_320"
        self.n_heads = len(out_dims_head)
        super(MultiHeadResNet200D, self).__init__()
        
        # # load base model
        base_model = timm.create_model(self.base_name, num_classes=sum(out_dims_head), pretrained=False)
        in_features = base_model.num_features
        
        if pretrained:
            pretrained_model_path = CFG.student
            state_dict = dict()
            for k, v in torch.load(pretrained_model_path, map_location='cpu')["model"].items():
                if k[:6] == "model.":
                    k = k.replace("model.", "")
                state_dict[k] = v
            base_model.load_state_dict(state_dict)
        
        # # remove global pooling and head classifier
        base_model.reset_classifier(0, '')
        
        # # Shared CNN Bacbone
        self.backbone = base_model
        
        # # Multi Heads.
        for i, out_dim in enumerate(out_dims_head):
            layer_name = f"head_{i}"
            layer = nn.Sequential(
                SpatialAttentionBlock(in_features, [64, 32, 16, 1]),
                nn.AdaptiveAvgPool2d(output_size=1),
                nn.Flatten(start_dim=1),
                nn.Linear(in_features, in_features),
                nn.ReLU(inplace=True),
                nn.Dropout(0.5),
                nn.Linear(in_features, out_dim))
            setattr(self, layer_name, layer)

    def forward(self, x):
        h = self.backbone(x)
        hs = [getattr(self, f"head_{i}")(h) for i in range(self.n_heads)]
        y = torch.cat(hs, axis=1)
        return None, None, y

In [42]:
def valid_one_epoch(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    trues = []
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model(images)
            y_preds = y_preds.squeeze(1)
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        # record accuracy
        trues.append(labels.to('cpu').numpy())
        preds.append(y_preds.sigmoid().to('cpu').numpy())
        if CFG.accum_iter > 1:
            loss = loss / CFG.accum_iter
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            cusprint('EVAL: [{0}/{1}] '
                'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                'Elapsed {remain:s} '
                'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                .format(
                step, len(valid_loader), batch_time=batch_time,
                data_time=data_time, loss=losses,
                remain=timeSince(start, float(step+1)/len(valid_loader)),
                ))

    trues = np.concatenate(trues)
    predictions = np.concatenate(preds)
    return losses.avg, predictions, trues

# loss & optimizer & scheduler

In [43]:
class GradualWarmupSchedulerV3(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV3, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch >= self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

# Inference

In [44]:
# Load checkpoint
PATH='/root/workspace/RSNA2022RAWE/resnet50d_401_fold0_epoch11.pth'
if torch.cuda.is_available():
    checkpoint = torch.load(PATH)
else:
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))

# Load states
model.load_state_dict(checkpoint['model'])

# Evaluation mode
model.eval()
model.to(device)


RSNAClassifier(
  (model): ResNet(
    (conv1): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [45]:
test_dataset = TestDataset(test_df, transform=get_transforms(data='valid'))
test_loader = DataLoader(test_dataset, batch_size=CFG.valid_bs, shuffle=False,num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

preds = []
for step, (images) in enumerate(test_loader):
    images = images.to(device, dtype=torch.float)
    with torch.no_grad():
        y_preds = model(images)
        y_preds = y_preds.squeeze(1)
    preds.append(y_preds.sigmoid().to('cpu').numpy())
    
predictions = np.concatenate(preds)

test_df['Preds'] = predictions * 100

test_df.to_csv(f"{outputdir}/test_preds.csv", index=False)